In [7]:
import pandas as pd
dataset_url='https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney'
tvshows = pd.read_csv("tv_shows.csv")

In [ ]:
pip install pandas


# READING THE DATASET

In [2]:
tvshows

,Unnamed: 0,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type
0,0,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,1
1,1,Stranger Things,2016,16+,8.8,93%,1,0,0,0,1
2,2,Money Heist,2017,18+,8.4,91%,1,0,0,0,1
3,3,Sherlock,2010,16+,9.1,78%,1,0,0,0,1
4,4,Better Call Saul,2015,18+,8.7,97%,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
5606,5606,Tut's Treasures: Hidden Secrets,2018,NaN,NaN,NaN,0,0,0,1,1
5607,5607,Paradise Islands,2017,NaN,NaN,NaN,0,0,0,1,1
5608,5608,Wild Russia,2018,NaN,NaN,NaN,0,0,0,1,1
5609,5609,Love & Vets,2017,NaN,NaN,NaN,0,0,0,1,1


# CLEANING THE DATASET

In [8]:
import re
def clean_title(title):
    return re.sub("^a-zA-Z0-9 ]","",title)
    

In [9]:
tvshows["clean_title"]=tvshows["Title"].apply(clean_title)

In [21]:
tvshows


,Unnamed: 0,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type,clean_title
0,0,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,1,Breaking Bad
1,1,Stranger Things,2016,16+,8.8,93%,1,0,0,0,1,Stranger Things
2,2,Money Heist,2017,18+,8.4,91%,1,0,0,0,1,Money Heist
3,3,Sherlock,2010,16+,9.1,78%,1,0,0,0,1,Sherlock
4,4,Better Call Saul,2015,18+,8.7,97%,1,0,0,0,1,Better Call Saul
...,...,...,...,...,...,...,...,...,...,...,...,...
5606,5606,Tut's Treasures: Hidden Secrets,2018,NaN,NaN,NaN,0,0,0,1,1,Tut's Treasures: Hidden Secrets
5607,5607,Paradise Islands,2017,NaN,NaN,NaN,0,0,0,1,1,Paradise Islands
5608,5608,Wild Russia,2018,NaN,NaN,NaN,0,0,0,1,1,Wild Russia
5609,5609,Love & Vets,2017,NaN,NaN,NaN,0,0,0,1,1,Love & Vets


# CREATING TFIDF MATRIX

In [22]:
tvshows["Year"]

0       2008
1       2016
2       2017
3       2010
4       2015
        ... 
5606    2018
5607    2017
5608    2018
5609    2017
5610    2016
Name: Year, Length: 5611, dtype: int64

In [ ]:
pip install scikit-learn

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(ngram_range=(1,2))
tfidf= vectorizer.fit_transform(tvshows["clean_title"])

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def search(title):
    title=clean_title(title)
    query_vec=vectorizer.transform([title])
    similarity= cosine_similarity(query_vec,tfidf).flatten()
    indices=np.argpartition(similarity,-5)[-5:]
    results=tvshows.iloc[indices][::-5]
    return results

In [24]:
search("Sherlock")

,Unnamed: 0,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type,clean_title
5610,5610,United States of Animals,2016,NaN,NaN,NaN,0,0,0,1,1,United States of Animals


# INTERACTIVE SEARCH BOX FUNCTION

In [25]:
import ipywidgets as widgets
from IPython.display import display

tv_show_input= widgets.Text(
   value="Stranger Things",
   description="Movie Title",
   disabled=False
)

tv_show_list=widgets.Output()

def on_type(data):
    with tv_show_list:
        tv_show_list.clear_output()
        title=data["new"]
        if len(title)>5:
            display(search(title))

tv_show_input.observe(on_type, names="value")

display(tv_show_input, tv_show_list)

Text(value='Stranger Things', description='Movie Title')

Output()

In [22]:
tvshows["Age"]

0       18+
1       16+
2       18+
3       16+
4       18+
       ... 
5606    NaN
5607    NaN
5608    NaN
5609    NaN
5610    NaN
Name: Age, Length: 5611, dtype: object

In [21]:
tvshows["IMDb"]

0       9.5
1       8.8
2       8.4
3       9.1
4       8.7
       ... 
5606    NaN
5607    NaN
5608    NaN
5609    NaN
5610    NaN
Name: IMDb, Length: 5611, dtype: float64

In [28]:
similar_users=tvshows[(tvshows["IMDb"]>7) &(tvshows["clean_title"]=="Stranger Things")]

In [29]:
similar_users

,Unnamed: 0,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type,clean_title
1,1,Stranger Things,2016,16+,8.8,93%,1,0,0,0,1,Stranger Things


## Merging all TV Shows Datasets

In [15]:
import pandas as pd
N_data= pd.read_csv("netflix_titles.csv")

In [16]:
N_data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [32]:
prime_data=pd.read_csv('amazon_prime_titles.csv')

In [33]:
prime_data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...
...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve..."
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."


In [53]:
data=pd.concat([prime_data,N_data], keys=["Prime", "Netflix"])

In [55]:
data

show_id     type                 title         director  \
Prime   0         s1    Movie   The Grand Seduction     Don McKellar   
        1         s2    Movie  Take Care Good Night     Girish Joshi   
        2         s3    Movie  Secrets of Deception      Josh Webber   
        3         s4    Movie    Pink: Staying True   Sonia Anderson   
        4         s5    Movie         Monster Maker     Giles Foster   
...              ...      ...                   ...              ...   
Netflix 8802   s8803    Movie                Zodiac    David Fincher   
        8803   s8804  TV Show           Zombie Dumb              NaN   
        8804   s8805    Movie            Zombieland  Ruben Fleischer   
        8805   s8806    Movie                  Zoom     Peter Hewitt   
        8806   s8807    Movie                Zubaan      Mozez Singh   

                                                           cast  \
Prime   0        Brendan Gleeson, Taylor Kitsch, Gordon Pinsent   
        1      Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar   
        2     Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...   
        3     Interviews with: Pink, Adele, Beyoncé, Britney...   
        4     Harry Dean Stanton, Kieran O'Brien, George Cos...   
...                                                         ...   
Netflix 8802  Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...   
        8803                                                NaN   
        8804  Jesse Eisenberg, Woody Harrelson, Emma Stone, ...   
        8805  Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...   
        8806  Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...   

                     country         date_added  release_year rating  \
Prime   0             Canada     March 30, 2021          2014    NaN   
        1              India     March 30, 2021          2018    13+   
        2      United States     March 30, 2021          2017    NaN   
        3      United States     March 30, 2021          2014    NaN   
        4     United Kingdom     March 30, 2021          1989    NaN   
...                      ...                ...           ...    ...   
Netflix 8802   United States  November 20, 2019          2007      R   
        8803             NaN       July 1, 2019          2018  TV-Y7   
        8804   United States   November 1, 2019          2009      R   
        8805   United States   January 11, 2020          2006     PG   
        8806           India      March 2, 2019          2015  TV-14   

               duration                                       listed_in  \
Prime   0       113 min                                   Comedy, Drama   
        1       110 min                            Drama, International   
        2        74 min                         Action, Drama, Suspense   
        3        69 min                                     Documentary   
        4        45 min                                  Drama, Fantasy   
...                 ...                                             ...   
Netflix 8802    158 min                  Cult Movies, Dramas, Thrillers   
        8803  2 Seasons          Kids' TV, Korean TV Shows, TV Comedies   
        8804     88 min                         Comedies, Horror Movies   
        8805     88 min              Children & Family Movies, Comedies   
        8806    111 min  Dramas, International Movies, Music & Musicals   

                                                    description  
Prime   0     A small fishing village must procure a local d...  
        1     A Metro Family decides to fight a Cyber Crimin...  
        2     After a man discovers his wife is cheating on ...  
        3     Pink breaks the mold once again, bringing her ...  
        4     Teenage Matt Banting wants to work with a famo...  
...                                                         ...  
Netflix 8802  A political cartoonist, a crime reporter and a...  
        8803  While living alone in a spooky town

In [56]:
hulu_data=pd.read_csv('hulu_titles.csv')

In [57]:
data=pd.concat([prime_data,N_data,hulu_data], keys=["Prime", "Netflix","Hulu"])

In [71]:
data.to_csv("tv_shows_data.csv",index=False)

In [72]:
disney_data=pd.read_csv('disney_plus_titles.csv')

In [73]:
data=pd.concat([prime_data,N_data,hulu_data,disney_data], keys=["Prime", "Netflix","Hulu","Disney Plus"])

In [74]:
data.to_csv("tv_shows_data.csv",index=False)

In [75]:
data

show_id   type  \
Prime       0         s1  Movie   
            1         s2  Movie   
            2         s3  Movie   
            3         s4  Movie   
            4         s5  Movie   
...                  ...    ...   
Disney Plus 1445   s1446  Movie   
            1446   s1447  Movie   
            1447   s1448  Movie   
            1448   s1449  Movie   
            1449   s1450  Movie   

                                                           title  \
Prime       0                                The Grand Seduction   
            1                               Take Care Good Night   
            2                               Secrets of Deception   
            3                                 Pink: Staying True   
            4                                      Monster Maker   
...                                                          ...   
Disney Plus 1445                        X-Men Origins: Wolverine   
            1446  Night at the Museum: Battle of the Smithsonian   
            1447                                 Eddie the Eagle   
            1448                            Bend It Like Beckham   
            1449           Captain Sparky vs. The Flying Saucers   

                         director  \
Prime       0        Don McKellar   
            1        Girish Joshi   
            2         Josh Webber   
            3      Sonia Anderson   
            4        Giles Foster   
...                           ...   
Disney Plus 1445       Gavin Hood   
            1446       Shawn Levy   
            1447  Dexter Fletcher   
            1448  Gurinder Chadha   
            1449      Mark Waring   

                                                               cast  \
Prime       0        Brendan Gleeson, Taylor Kitsch, Gordon Pinsent   
            1      Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar   
            2     Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...   
            3     Interviews with: Pink, Adele, Beyoncé, Britney...   
            4     Harry Dean Stanton, Kieran O'Brien, George Cos...   
...                                                             ...   
Disney Plus 1445  Hugh Jackman, Liev Schreiber, Danny Huston, wi...   
            1446  Ben Stiller, Amy Adams, Owen Wilson, Hank Azar...   
            1447  Tom Costello, Jo Hartley, Keith Allen, Dickon ...   
            1448  Parminder Nagra, Keira Knightley, Jonathan Rhy...   
            1449                                      Charlie Tahan   

                                                 country          date_added  \
Prime       0                                     Canada      March 30, 2021   
            1                                      India      March 30, 2021   
            2                              United States      March 30, 2021   
            3                              United States      March 30, 2021   
            4                             United Kingdom      March 30, 2021   
...                                                  ...                 ...   
Disney Plus 1445           United States, United Kingdom        June 4, 2021   
            1446                   United States, Canada       April 2, 2021   
            1447  United Kingdom, Germany, United States   December 18, 2020   
            1448  United Kingdom, Germany, United States  September 18, 2020   
            1449                           United States       April 1, 2020   

                  release_year rating duration  \
Prime       0             2014    NaN  113 min   
            1             2018    13+  110 min   
            2             2017    NaN   74 min   
            3             2014    NaN   69 min   
            4             1989    NaN   45 min   
...                        ...    ...      ...   
Disney Plus 1445          2009  PG-13  108 min   
            1446          2009     PG  106 min   
            1447          2016  PG-13  107 min   
            1448    